In [ ]:
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import spacy
from spacy.matcher import Matcher
from spacy import displacy
import pandas as pd
import numpy as np
import re
import multiprocessing as mp

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.0 MB/s 
     |████████████████████████████████| 636 kB 39.6 MB/s 
     |████████████████████████████████| 895 kB 29.0 MB/s 


In [ ]:
%%capture
!python -m spacy download en_core_web_lg -q
import en_core_web_lg 
nlp=en_core_web_lg.load()
matcher = Matcher(nlp.vocab)

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# postags=nltk.pos_tag(tok)
# postags

### uncertainity

#### model verbs

In [ ]:
def to_nlp_tags(text):
  doc=nlp(text)
  sents=list(doc.sents)
  return pd.Series([doc,sents], index=['doc', 'sents'])    

In [ ]:
# for spacy need to install below
# !python -m spacy download en_core_web_lg
# import en_core_web_lg 
# nlp=en_core_web_lg.load()
# matcher = Matcher(nlp.vocab)
def get_MD_verb_count(sents,percentage=True,lib="spacy"):
  if lib=="spacy":
    # doc=nlp(text)
    # sents = list(doc.sents)
    # dep=[]
    # tag=[]
    count=0
    for sent in sents:
      for token in sent:
        # dep.append(token.dep_)
        # tag.append(token.tag_)
        if token.tag_ =="MD":
          count=count+1

    return count/len(sents) if percentage else count

  else: ## nltk
    tok=nltk.word_tokenize(text.lower())
    postags=nltk.pos_tag(tok)
    return len([i[0] for i in postags if i[1]=="MD"])/len(tok) if percentage else len([i[0] for i in postags if i[1]=="MD"])


# text="You must obey the rules, or you will get puniched"
# get_MD_verb_count(to_nlp_tags(text))

#### qn marks

In [ ]:
# def get_no_of_qn_marks(text,percentage=False):
#   return text.count("?")/len(text) if percentage else text.count("?")

#### quantifiers

In [ ]:
def count_quantifiers(sents ,nlp=nlp ,matcher=matcher,percentage=False):
  # doc=nlp(text)
  # sents = list(doc.sents)
  # dep=[]
  # tag=[]
  count=0
  for sent in sents:
    for token in sent:
      # dep.append(token.dep_)
      # tag.append(token.tag_)
      if token.tag_ =="CD":
        # print(token)
        count=count+1

  return count/len(sents) if percentage else count

In [ ]:
# count_quantifiers("thirteen fifteen sixty three few many year old ")

### non_immediacy

In [ ]:
# run below commented section to downlaod spacy corpous 
# !python -m spacy download en_core_web_lg
# https://gist.github.com/armsp/30c2c1e19a0f1660944303cf079f831a
# import en_core_web_lg 
# nlp=en_core_web_lg.load()
# matcher = Matcher(nlp.vocab)
def count_passive(sents ,nlp=nlp ,matcher=matcher):
  # doc = nlp(text)
  # sents = list(doc.sents)
  # print("Number of Sentences = ",len(sents))
  # for sent in doc.sents:
  #   for token in sent:
  #       print(token.dep_,token.tag_, end = " ")
  #   print()
  passive_rule = [{'DEP':'nsubjpass'},{'DEP':'aux','OP':'*'},{'DEP':'auxpass'},{'TAG':'VBN'}]
  matcher.add('Passive',None,passive_rule)
  matches = matcher(doc)
  # print("Number of PASSIVE Sentences = " , len(matches))
  return len(matches)

In [ ]:
def non_immediacy(doc):
  FPS = ["i"	,"me",	"my"	,"mine",	"myself"]  # first person singular
  FPP = ["we"	,"us",	"our",	"ours",	"ourselves"] #first person pluran
  SPS =["you",	"your",	"yours",	"yourself"] # second person singular
  # SPP =["you",	"your",	"yours",	"yourself"] # second person plural
  TPS = ["he",	"him",	"his",	"himself" ,"she"	,"her",	"hers"	,"herself","it"	,"its",	"itself"]  # third person singular
  TPP=["they",	"them",	"their"	,"theirs"	,"themselves"] # third person plural

  # doc=nlp(text.lower())
  sents=list(doc.sents)

  dic={"fps":0,"fpp":0,"sps":0,"spp":0,"tps":0 ,"tpp":0}
  qt=False
  qtCount=0
  for sent in sents:
    for token in sent :
      strVal=token.string.lower()
      if (token.tag_=="``"):
        if qt:
          qtCount+=1
        qt=!qt
      if  strVal in FPS :
        dic["fps"]+=1
      elif strVal in FPP:
        dic["fpp"]+=1
      elif strVal in SPS:
        dic["sps"]+=1
      elif strVal in TPS:
        dic["tps"]+=1
      elif strVal in TPP:
        dic["tpp"]+=1
  dic["passive_count"]=count_passive(doc)
  dic["sentence_count"]=len(sents)
  dic["quated_text"]=qtCount
  # return dic
  return pd.Series([dic["passive_count"],dic["sentence_count"],dic["quated_text"],dic["fps"],dic["fpp"],dic["sps"],dic["tps"],dic["tpp"]], index=['passive_count', 'sentence_count', 'quated_text','fps','fpp','sps','tps','tpp'])
      
      



### Example

In [ ]:
# Passive = "A camera is bought by him.\
# Water is drunk by her.\
# He is known to me.\
# A tub is filled with water.\
# Sugar is sold in kilograms.\
# There is a considerable range of expertise demonstrated by the spam senders.\
# It was determined by the committee that the report was inconclusive.\
# The metal beams were eventually corroded by the saltwater.\
# The baby was carried by the kangaroo in her pouch.\
# The last cookie was eaten by whom?"
# count_passive(Passive)

In [ ]:
# nlp=en_core_web_lg.load()
doc=nlp("\"you'll never do this\", said Fahir")

In [ ]:
displacy.render(doc, style="dep" ,jupyter=True)
# displacy.render(doc, style="ent" ,jupyter=True)

In [ ]:
sent= list(doc.sents)[0]
for token in sent:
    print(token,token.dep_,token.tag_,token.tag_=="``") 

" punct `` True
you nsubj PRP False
'll aux MD False
never neg RB False
do ccomp VB False
this dobj DT False
" punct '' False
, punct , False
said ROOT VBD False
Fahir nsubj NNP False


In [ ]:
print("Pipeline:", nlp.pipe_names)

Pipeline: ['tagger', 'parser', 'ner']


In [ ]:
# df.to_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/WELFake.csv")

### Df

In [ ]:
# df=pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/Nilaan.csv")

In [ ]:
# df.head()

In [ ]:
# df.info()

In [ ]:
# df = df.dropna(subset=['text'], how='all')
# df.info()
# df = df.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72095 entries, 0 to 72133
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    72095 non-null  int64 
 1   Unnamed: 0.1  72095 non-null  int64 
 2   title         71537 non-null  object
 3   text          72095 non-null  object
 4   label         72095 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 3.3+ MB


In [ ]:
# df['text'] = df['text'].replace(np.nan, '', regex=True)

In [ ]:
# def url_count(text):
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     urls=re.findall(url_pattern,text)
#     return len(urls)

In [ ]:
# df["url_count"]=df["text"].apply(url_count)

In [ ]:
# # remove urls
# def remove_urls(text):
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     return url_pattern.sub(r'', text)

In [ ]:
# df["text"]=df["text"].apply(remove_urls)

In [ ]:
# def remove_nonascii(sent):
#   return " ".join(("".join([i for i in sent if i.isascii()])).split())

In [ ]:
# df["text"]=df["text"].apply(remove_nonascii)

In [ ]:
# df = df.dropna(subset=['text'], how='all')
# df.info()
# df = df.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72095 entries, 0 to 72094
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    72095 non-null  int64 
 1   Unnamed: 0.1  72095 non-null  int64 
 2   title         71537 non-null  object
 3   text          72095 non-null  object
 4   label         72095 non-null  int64 
 5   url_count     72095 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 3.9+ MB


In [ ]:
# df["qn_symbol"]=df["text"].apply(get_no_of_qn_marks)
# df["len"]=df["text"].apply(lambda x:len(x.split()))

In [ ]:
# df.describe()

In [ ]:
# df=df[df["qn_symbol"]<10]
# df=df[df["url_count"]<10]
# df=df[df["len"]>5]
# df=df[df["len"]<1000]

In [ ]:
# def detect_lang(text):
#   try:
#       return detect(text)
#   except:
#       return 'error'

In [ ]:
# ! pip install langdetect -q
# from langdetect import detect
# # from langdetect import DetectorFactory
# # DetectorFactory.seed = 0
# df["lang"]=df["text"].apply(detect_lang)

In [ ]:
# df=df[df["lang"]=='en']

In [ ]:
df=pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/sementic+sentiment_welfake.csv")
df=df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
df=df.drop(columns="len" )

duplicate_count = len(df['text'])-len(df['text'].drop_duplicates())
print('Duplicate count:', duplicate_count)
print('Total records before remove duplicates:', df.shape[0])

# drop duplicates (keep the last post of each of the duplicates)

df = df.drop_duplicates(subset='text', keep="first")
print('updated record count:', df.shape[0])

Duplicate count: 0
Total records before remove duplicates: 54356
updated record count: 54356


In [ ]:
# df=df.sample(100)

In [ ]:
df.head()

,title,text,label,url_count,qn_symbol,lang,num_chars,num_words,num_sentences,words_per_sentence,characters_per_word,punctuations_per_sentence,positive,negative,num_exclamation,get_sentiment_polarity,lexical_diversity,content_word_diversity_and_redundancy,nvaa,content_word_diversity,redundancy,noun,verb,adj,adv,content,14_emo_dict,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,8_emo_dict,highest_14_emo_label,highest_8_emo_label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,0,7,en,4979,870,44,19.772727,4.427130,4.136364,1.457399,1.457399,2,-0.9926,42.528736,"{'content_word_diversity': 31.494252873563216,...","{'NOUN': 12.298850574712644, 'VERB': 10.919540...",31.494253,13.218391,12.298851,10.919540,4.252874,3.218391,No comment is expected from Barack Obama Membe...,"{'fear': 0.005, 'senerity': 0.008, 'distractio...",0.131,0.047,0.046,0.005,0.445,0.024,0.016,0.327,"{'sadness': 0.024, 'anger': 0.131, 'joy': 0.44...",joy_ecstasy,joy
1,NaN,Did they post their votes for Hillary already?,1,0,1,en,46,8,1,8.000000,4.750000,1.000000,0.000000,0.000000,0,0.0000,100.000000,"{'content_word_diversity': 37.5, 'redundancy':...","{'NOUN': 12.5, 'VERB': 25.0, 'ADJ': 0.0, 'ADV'...",37.500000,62.500000,12.500000,25.000000,0.000000,12.500000,Did they post their votes for Hillary already?,"{'sadness': 0.015, 'fear': 0.021, 'trust': 0.0...",0.060,0.133,0.261,0.021,0.221,0.015,0.126,0.203,"{'sadness': 0.015, 'anger': 0.06, 'joy': 0.221...",boredom,disgust
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last n...",1,0,0,en,212,34,2,17.000000,5.205882,1.000000,5.882353,5.882353,0,0.7351,91.176471,"{'content_word_diversity': 58.82352941176471, ...","{'NOUN': 23.52941176470588, 'VERB': 17.6470588...",58.823529,32.352941,23.529412,17.647059,11.764706,0.000000,"Now, most of the demonstrators gathered last n...","{'interest_vigilance': 0.029, 'distraction': 0...",0.053,0.089,0.184,0.033,0.265,0.000,0.066,0.350,"{'sadness': 0, 'anger': 0.053, 'joy': 0.265, '...",acceptance,trust
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,0,0,en,1904,326,10,32.600000,4.760736,4.200000,1.840491,1.840491,0,-0.9081,53.374233,"{'content_word_diversity': 41.717791411042946,...","{'NOUN': 13.496932515337424, 'VERB': 9.8159509...",41.717791,13.496933,13.496933,9.815951,5.214724,1.840491,"The RS-28 Sarmat missile, dubbed Satan 2, will...","{'senerity': 0.008, 'fear': 0.02, 'acceptance'...",0.627,0.035,0.193,0.020,0.072,0.000,0.067,0.026,"{'sadness': 0, 'anger': 0.627, 'joy': 0.072000...",anger,anger
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,0,0,en,1519,244,10,24.400000,5.065306,4.100000,2.448980,2.448980,1,-0.9255,58.606557,"{'content_word_diversity': 42.21311475409836, ...","{'NOUN': 17.21311475409836, 'VERB': 11.4754098...",42.213115,19.262295,17.213115,11.475410,6.557377,2.459016,All we can say on this one is it s about time ...,"{'fear': 0.02, 'interest_vigilance': 0.025, 'a...",0.544,0.025,0.324,0.020,0.000,0.089,0.039,0.000,"{'sadness': 0.089, 'anger': 0.544, 'joy': 0, '...",anger,anger


In [ ]:
# df.describe()

In [ ]:
#   # doc=nlp(text.lower())
#   # sents=list(doc.sents)
# dg=df["text"].apply(to_nlp_tags) 

In [ ]:
# dff=df.sample(100)

In [ ]:
import multiprocessing as mp

In [ ]:
mp.cpu_count()

2

In [ ]:
# with mp.Pool(mp.cpu_count()) as pool:
#     dg = pool.map(to_nlp_tags, df['text'])

In [ ]:
dg =df["text"].progress_apply(to_nlp_tags)

  0%|          | 0/54356 [00:00<?, ?it/s]

In [ ]:
df=pd.concat([df,dg],axis=1)

In [ ]:
dgg=df["doc"].progress_apply(non_immediacy)

In [ ]:
df=pd.concat([df,dgg],axis=1)

In [ ]:
df["model_verbs"]=df["sents"].progress_apply(get_MD_verb_count)

In [ ]:
df["quantifiers"]=df["sents"].progress_apply(count_quantifiers)

In [ ]:
df.to_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/Nilaan5.csv")

In [ ]:
df.to_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/Nilaan1.csv")

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/Nilaan5.csv")